In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D 
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
# from alexnet import AlexNet

ImportError: cannot import name 'BatchNormalization' from 'keras.layers.normalization' (d:\1-programme\porgramme\anaconda\envs\test1\lib\site-packages\keras\layers\normalization\__init__.py)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data() #load dataset

In [ ]:
#preprocess images
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

In [ ]:
#Alexnet model for feature extraction

model = Sequential()
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(28, (5, 5),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(28, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10))
model.add(BatchNormalization())
model.add(Activation('softmax'))
 
    

In [ ]:
#compile model
model.compile(optimizer='rmsprop', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=5)

In [ ]:
#get features from last layer for traning data
get_7th_layer_output = K.function([model.layers[0].input],
                                  [model.layers[6].output])
# layer_output = get_3rd_layer_output([x_train])[0]

In [ ]:
layer_output_train = get_7th_layer_output([x_train])

In [ ]:
print(layer_output_train[0].shape)
layer_output_train =layer_output_train[0]

In [ ]:
#get features from last layer for test data
get_7th_layer_test_output = K.function([model.layers[0].input],
                                  [model.layers[6].output])
layer_output_test = get_7th_layer_test_output([x_test])[0]
x_test.shape

In [ ]:
#reshape and convert training features to array
train_feature=[]
for l in layer_output_train:
    train_feature.append(l.reshape(1372))
train_feature = np.array(train_feature)
train_feature.shape

In [ ]:
#reshape and convert test features to array
test_feature=[]

for l in layer_output_test:
    test_feature.append(l.reshape(1372))
test_feature = np.array(test_feature)
test_feature.shape#get features from last layer for traning data

In [ ]:
#MLPClassifer model for training

Model = MLPClassifier(activation='relu', hidden_layer_sizes=(64, 64), alpha = 0.1)
# Model.fit(train_feature, y_train)
history = Model.fit(train_feature, y_train)

In [ ]:
#accuracy obtained with MLP as training model and alexnet as feature extraction model
Model.score(test_feature, y_test)
print("Training Score :: {}\n".format(Model.score(train_feature, y_train)))
print("Testing Score :: {}\n".format(Model.score(test_feature, y_test)))